<a href="https://colab.research.google.com/github/Anurag17singh/Machine-Learning-and-Deep-Learning/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [71]:
df=pd.read_csv('/content/drive/MyDrive/Copy of Evaluation-dataset.csv')
df.head(5)

,"Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience.",garage service positive,ease of booking positive,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,"Easy Tyre Selection Process, Competitive Prici...",garage service positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Very easy to use and good value for money.,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Really easy and convenient to arrange,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,It was so easy to select tyre sizes and arrang...,location positive,value for money positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,service was excellent. Only slight downside wa...,length of fitting positive,ease of booking positive,ease of booking negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
data=df.copy()
data.drop(['Unnamed: 3',
 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
 'Unnamed: 14'],axis=1,inplace=True)

In [73]:
data.columns

Index(['Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience.',
       'garage service positive', 'ease of booking positive'],
      dtype='object')

In [74]:
data.columns=['Review','Garage Service','Ease of booking']

In [75]:
most_frequent = data['Garage Service'].mode()[0]
data['Garage Service'] = data['Garage Service'].fillna(most_frequent)

In [76]:
most_frequent = data['Ease of booking'].mode()[0]
data['Ease of booking'] = data['Ease of booking'].fillna(most_frequent)

In [77]:
data.isnull().sum()

Review             0
Garage Service     0
Ease of booking    0
dtype: int64

In [78]:
# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment
def get_sentiment(text):
    # Get the sentiment scores dictionary
    sentiment_scores = sid.polarity_scores(str(text))
    # Return the compound score
    return sentiment_scores['compound']

# Define a function to convert polarity into sentiment category
def get_sentiment_category(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Assuming 'df' is your DataFrame and 'label' is the column with the correct sentiment labels
# Preprocess the data
label=[]
for column in ['Review', 'Garage Service', 'Ease of booking']:
    data[column + '_sentiment_score'] = data[column].apply(get_sentiment)
    data[column + '_sentiment_category'] = data[column + '_sentiment_score'].apply(get_sentiment_category)
    label.append( data[column + '_sentiment_score'])

In [80]:
data[label[0].name] = label[0]
print(data)

                                                  Review  \
0      Easy Tyre Selection Process, Competitive Prici...   
1             Very easy to use and good value for money.   
2                  Really easy and convenient to arrange   
3      It was so easy to select tyre sizes and arrang...   
4      service was excellent. Only slight downside wa...   
...                                                  ...   
10126  I ordered the wrong tyres, however [REDACTED] ...   
10127  Good experience, first time I have used [REDAC...   
10128  I ordered the tyre I needed on line, booked a ...   
10129  Excellent service from point of order to fitti...   
10130  Seamless, well managed at both ends. I would r...   

                   Garage Service                Ease of booking  \
0         garage service positive       value for money positive   
1        value for money positive       value for money positive   
2        ease of booking positive       value for money positive   
3      

In [81]:
data['score'] = data[['Review_sentiment_score', 'Garage Service_sentiment_score', 'Ease of booking_sentiment_score']].mean(axis=1)

In [82]:
data['score'].head()

0    0.694400
1    0.751367
2    0.646000
3    0.646667
4    0.616067
Name: score, dtype: float64

In [83]:
data['score_category'] = pd.cut(data['score'], bins=3, labels=['low', 'medium', 'high'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data[['Review_sentiment_score', 'Garage Service_sentiment_score', 'Ease of booking_sentiment_score']], data['score_category'], test_size=0.2, random_state=42)
# For this example, we'll use a simple logistic regression model

# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)



LogisticRegression()

In [84]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.9925999013320178


In [85]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

        high       1.00      1.00      1.00      1768
         low       1.00      0.94      0.97        80
      medium       0.96      0.96      0.96       179

    accuracy                           0.99      2027
   macro avg       0.99      0.96      0.97      2027
weighted avg       0.99      0.99      0.99      2027



*It is a Sentiment Analysis task of Evaluation.csv dataset.
The dataset is not labelled data means it not contains output, it is unsupervised data.From 15 features only three features are perfect.Means in other features more than 90% or some of them 100% contains nan values.So I need to drop it and process only 3 features.For processing I have use SentimentIntensityAnalyzer. For model building i have use logistic regression algorithm.*